In [1]:
from collections import defaultdict
from random import choice, randint, random
import time

In [2]:
operand_range = list(range(1, 11))

In [3]:
class Player:
    def __init__(self):
        self.history = defaultdict(list)
    
    def update(self, question, answer):
        self.history[question].append(answer)
        
    def answer(self, question):
        if question in self.history:
            return choice(self.history[question])
        else:
            return Answer(choice(operand_range), 5*(1 + random()))
    
    def __repr__(self):
        return '\n'.join([f'{question}: {" ".join([str(answer) for answer in answers])}' for question, answers in self.history.items()])

In [4]:
class Answer:
    def __init__(self, answer, time_taken):
        self.answer = answer
        self.time_taken = time_taken
        
    def __repr__(self):
        return f'{self.answer} in {self.time_taken:0.3f}s'

In [5]:
class Question:
    def __init__(self, operation, first_operand, second_operand):
        self.operation = operation
        self.first_operand = first_operand
        self.second_operand = second_operand
    
    def __repr__(self):
        return ' '.join([
            str(self.first_operand),
            self.operation,
            str(self.second_operand)
        ])
    
    def __eq__(self, other):
        return self.__hash__() == hash(other)
    
    def get_answer(self):
        return eval(str(self))
    
    def __hash__(self):
        return hash((self.operation, *sorted((self.first_operand, self.second_operand))))

In [6]:
class QuestionGenerator:
    def __init__(self, operation, main_operand=None, min_operand=None, max_operand=None):
        self.operation = operation
        if main_operand:
            self.main_operand = main_operand
        else:
            self.operand_range = list(range(self.min_operand, max_operand + 1))
    
    def get(self):
        second_operand = choice(operand_range)
        if self.main_operand:
            return Question(self.operation, self.main_operand, second_operand)
        else:
            return Question(self.operation, choice(operand_range), second_operand)

In [7]:
class TrainingSession:
    def __init__(self, player, n_questions, operation, main_operand=None, min_operand=None, max_operand=None):
        self.n_questions = n_questions
        self.questionGenerator = QuestionGenerator(operation, main_operand, min_operand, max_operand)
        self.player = player
    
    def train(self):
        for i in range(self.n_questions):
            question = self.questionGenerator.get()
            t0 = time.time()
            answer = int(input(f'{question} = '))
            t1 = time.time()
            correct = question.get_answer() == answer
            time_taken = t1 - t0
            self.player.update(question, Answer(answer, time_taken))
            print('Correct' if correct else 'Wrong', f'in {time_taken:0.3f}s')

In [8]:
class Competition:
    def __init__(self, player1, player2, n_questions, operation, operand):
        self.player1 = player1
        self.player2 = player2
        self.n_questions = n_questions
        self.questionGenerator = QuestionGenerator(operation, operand)

    def compete(self):
        for i in range(self.n_questions):
            question = self.questionGenerator.get()
            print(question)
            a1 = self.player1.answer(question)
            a2 = self.player2.answer(question)
            print('Player 1:', a1, '| Player 2:', a2)

In [9]:
p1 = Player()
p2 = Player()

In [10]:
TrainingSession(p1, 4, '*', 2).train()

2 * 1 = 2
Correct in 2.399s
2 * 2 = 4
Correct in 1.351s
2 * 6 = 12
Correct in 1.568s
2 * 3 = 6
Correct in 1.716s


In [11]:
TrainingSession(p2, 4, '*', 2).train()

2 * 10 = 20
Correct in 1.478s
2 * 6 = 12
Correct in 1.391s
2 * 1 = 2
Correct in 1.143s
2 * 5 = 10
Correct in 1.604s


In [12]:
c = Competition(p1, p2, 4, '*', 2)

In [13]:
c.compete()

2 * 2
Player 1: 4 in 1.351s | Player 2: 9 in 5.841s
2 * 4
Player 1: 4 in 6.893s | Player 2: 7 in 9.258s
2 * 1
Player 1: 2 in 2.399s | Player 2: 2 in 1.143s
2 * 2
Player 1: 4 in 1.351s | Player 2: 10 in 9.948s
